In [2]:
import numpy as np
import pandas as pd

# Metropolis-Hastings MCMC
A simple example: Binary Distribution

In [8]:
# true distribution
# we cant evaluate at any point z
p = [0.35, 0.65]

r = []
z_candidate, z = 0, 0
for _ in range(100000):
    r.append(z)
    # transition probability
    A = np.min([1, p[z_candidate] / p[z]])
    # accept of reject candidate
    if A > np.random.uniform(0, 1):
        # move to the candidate state
        z = z_candidate
    # sample new candidate from the markov chain
    # p = 0.5 because p(z_{t+1} | p_t) = p(z_t | p_{z+1})
    z_candidate = np.random.binomial(1, p=0.5)

# print distribution
pd.Series(r).value_counts() / len(r)

1    0.64659
0    0.35341
Name: count, dtype: float64

Multinomial distribution.

In [37]:
# transition probability matrix
# for Markov chain
T = np.ones((3, 3)) * 1/3

# true distribution
# we cant evaluate at any point z
p = [0.35, 0.25, 0.40]

r = []
z_candidate, z = 0, 0
for _ in range(100000):
    r.append(z)
    # p(z*) = 0.5
    A = np.min([1, p[z_candidate] / p[z]])
    # accept of reject candidate
    if A > np.random.uniform(0, 1):
        # move to the candidate state
        z = z_candidate
    # sample new candidate from the chain
    # p = 0.5 because p(z_{t+1} | p_t) = p(z_t | p_{z+1})
    z_candidate = np.random.multinomial(1, pvals=T[z]).argmax()

# print distribution
(pd.Series(r).value_counts() / len(r)).sort_index()

0    0.34922
1    0.24977
2    0.40101
Name: count, dtype: float64

## Gibbs Sampler

Special case of the metropolis-hastings algorithm. The joint distribution is given by:

$$
\prod_{t=1}^T P(\theta | \alpha) P(z_{t} | \theta) \prod_{l=1}^L  P(\Phi_{t}^{(l)} | \beta^{(l)}) P(f_t^{(l)} | z_t, \Phi_t^{(l)}) \\

\propto \prod_{t=1}^T \prod_{k=1}^K \theta_k^{z_{kt} + \alpha_k - 1} \prod_{l=1}^L  P(\Phi_{tk}^{(l)} | z_{tk}, \beta^{(l)}) P(f_t^{(l)} | z_t, \Phi_t^{(l)}, \beta^{(l)}) \\
$$

$$
P(\Phi_{t}^{(l)} | \beta^{(l)}) = \prod_{k=1}^K (\Phi_{tk}^{(l)})^{\beta_k - 1} \\

\prod_{t=1}^T P(\Phi_{t}^{(l)} | \beta^{(l)}) P(f_t^{(l)} | z_t, \Phi_t^{(l)}) 
    = \prod_{t=1}^T \prod_{k=1}^K \prod_{x=1}^V (\Phi_{tk}^{(l)})^{f_{tx}^{(l)} z_{tk} + \beta_k^{(l)} - 1}
    = \prod_{k=1}^K (\Phi_{tk}^{(l)})^{\sum_{t=1}^T \sum_{x=1}^V (f_{tx}^{(l)} z_{tk}) + \beta_k^{(l)} - 1}
$$

Marginalizing over $\Phi_{tk}^{(l)}$ yields:

$$
P(f^{(l)}, z_t | \beta^{(l)}) 
    = \frac{\prod_{k=1}^K \Gamma(n^{(l)}_{k} + \beta_k)}{\Gamma(n^{(l)} + \beta_0)}
$$

where $n^{(l)} = \sum_{k=1}^K n^{(l)}_k$ Then,

$$
P(f^{(l)}, z_t | z_{-t}, \beta^{(l)}) = \frac{P(f_t^{(l)}, z_t, z_{-t} | \beta^{(l)}) }{P(f_t^{(l)} z_{-t} | \beta^{(l)} ) }
    \propto \prod_{k=1}^K \frac{ \Gamma(n^{(l)}_k + \beta_k)}{\Gamma(n^{(l)}_{-tk} + \beta_k )} \frac{\Gamma(n^{(l)}_{-t} + \beta_0)}{\Gamma(n^{(l)} + \beta_0)}
    \propto \frac{n^{(l)}_{k} + \beta_k - z_{tk}}{n^{(l)} + \beta_0}
$$

where
$$
n^{(l)}_k = \sum_{t=1}^T \sum_{x=1}^V f_{tx}^{(l)} z_{tk} \\

n^{(l)}_{k, -(t, x)}= \sum_{t \neq t'}^T \sum_{x \neq x'}^V f_{tx}^{(l)} z_{tk} \\
$$

$$
P(\theta | \alpha) \propto \prod_{k=1}^K \theta_k^{\alpha_k - 1} \\

P(z_{t} | \theta) \propto \prod_{k=1}^K \theta_k^{z_{tk}}
$$

$$
P(\theta | \alpha) P(z_{tk} | \theta) \propto \prod_{k=1}^K \theta_k^{z_{kt} + \alpha_k - 1}
$$

and, 
$$
\prod_{t=1}^T P(\theta | \alpha) P(z_{t} | \theta) \propto \prod_{t=1}^T \prod_{k=1}^K \theta_k^{z_{tk} + \alpha_k - 1} 

    \propto \prod_{k=1}^K \theta_k^{n_{k} + \alpha_k - 1}
$$

Marginalizing over $\theta_k$ yeilds:

$$
P(z_t | \alpha) = \frac{\prod_{k=1}^K \Gamma(\alpha_k + z_{tk})}{\Gamma(\alpha_0 + 1)}
$$

Where $n_k = \sum_{t=1}^T z_{tk}$ Now,
$$
P(z_{t} | z_{-t}, \alpha) = \frac{P(z_{t}, z_{-t} | \alpha)}{P(z_{-t} | \alpha)} 
    = \prod_{k=1}^K \frac{\Gamma(\alpha_k + n_k)}{\Gamma(\alpha_k + n_{-tk})} \cdot \frac{\Gamma(\alpha_0 + N - 1)}{\Gamma(\alpha_0 + N)}
    \propto \prod_{k=1}^K \frac{\Gamma(\alpha_k + n_k)}{\Gamma(\alpha_k + n_{-tk})} 
    \propto \prod_{k=1}^K (\alpha_k + n_{tk} - z_{kt})
$$

Or just 
$$
P(z_{tk} | z_{-tk}, \alpha) \propto \alpha_k + n_{tk} - z_{tk}
$$

where $n_k$ is the number of datapoints $t$ assigned to cluster $k$.

Our "pseudo-counts" are the count of latent variable assignment to datapoints. 

$$
P(z_{i} | z_{-i})
$$



$$
P(z_t | f_t^{(l)}) = \frac{P(z_t, f_t^{(l)})}{\sum_{z_t} P(z_t, f_t^{(l)})}
$$

In [62]:
# suppose we have a dataset with 30 datapoints
# with the following cluster assignment counts
n = np.array([10, 20, 70])

r = []
z_candidate = 0
for _ in range(1000000):
    n[z_candidate] -= 1 # decrement
    # a-K = 1
    z_candidate = np.random.multinomial(1, pvals=(n+1)/(n+1).sum()).argmax()
    n[z_candidate] += 1 # increment

    r.append(z_candidate)


# print distribution
t = 100000
(pd.Series(r)[:t].value_counts() / len(r[:t])).sort_index()

0    0.09751
1    0.20606
2    0.69643
Name: count, dtype: float64

The joint distribution is given by:

$$
P(f, z, \Phi, \theta | \alpha, \beta)
    = \prod_{t=1}^T P(\theta | \alpha) P(z_{t} | \theta) \prod_{l=1}^L  P(\Phi^{(l)} | \beta^{(l)}) P(f_{t}^{(l)} | z_t, \Phi^{(l)}) \\
    = \prod_{k=1}^K \theta_k^{\sum_{t=1}^T z_{t} + \alpha_k - 1} \prod_{x=1}^V  (\Phi_{xk}^{(l)})^{\sum_{t=1}^T \sum_{x=1}^V f_{tx}^{(l)}z_t + \beta_k^{(l)} - 1} 
$$

Marginalizing over $\theta$ and $\Phi$ gives:

$$
    P(f, z | \alpha, \beta) = \int P(f, z, \Phi_t, \theta | \alpha, \beta) d\theta d\Phi \\
        = \frac{\prod_{k=1}^K \Gamma(n_k + \alpha_k)}{\Gamma(\sum_{k=1}^K n_k + \alpha_0)} 
            \frac{\prod_{x=1}^V \prod_{k=1}^K \Gamma(n_{xk} + \beta_k)}{\Gamma(\sum_{x=1}^V \sum_{k=1}^K n_{xk} + \beta_0)}
$$

Where we defined:

$$
    n_k \equiv \sum_{t=1}^T z_{tk} \\
    n_{xk} \equiv \sum_{t=1}^T f_{tx} z_{tk}
$$

Then, since $\Gamma(n+1) = n \Gamma(n)$:

$$
P(z_t | z_{-t}, f, \alpha, \beta) \propto P(f, z | \alpha, \beta) \\
    \propto \prod_{k \neq k'} \Gamma(n_{k, -t} + \alpha_k) 
        \prod_{x \notin f_t} \frac{\Gamma(n_{xk, -(t, x)} + \beta_k)}{\Gamma(\sum_{x=1}^V \sum_{k=1}^K n_{xk, -(t, x)} + \beta_0)}
        \Gamma(n_{k', -t} + \alpha_{k'} + 1) 
        \prod_{x \in f_t} \frac{\Gamma(n_{xk', -(t, x)} + \beta_k' + 1)}{\Gamma(\sum_{x=1}^V \sum_{k'=1}^K n_{xk', -(t, x)} + \beta_0 + 1)} \\
    = \prod_{k = 1}^K \Gamma(n_{k, -t} + \alpha_k) 
        \prod_{x=1}^V \frac{\Gamma(n_{xk, -(t, x)} + \beta_k)}{\Gamma(\sum_{x=1}^V \sum_{k=1}^K n_{xk, -(t, x)} + \beta_0)}
        (n_{k', -t} + \alpha_{k'}) \prod_{x \in f_t} \frac{n_{xk', -(t, x)} + \beta_{k'}}{\sum_{x=1}^V \sum_{k'=1}^K n_{xk', -(t, x)} + \beta_0} \\
    \propto (n_{k, -t} + \alpha_{k}) \prod_{x \in f_t}  \frac{n_{xk, -(t, x)} + \beta_k}{\sum_{x=1}^V \sum_{k'=1}^K n_{xk, -(t, x)} + \beta_0} \\
$$